## 1. Environment Setup and Library Installation

Initially, a new project folder was created and opened using the Command Prompt.

The following steps were followed to set up the environment:

1. Navigated to the project directory using:
cd <project-folder-path>

2. Created a virtual environment named `chatbot`:
chatbot\Scripts\activate

3. Activated the virtual environment:
chatbot\Scripts\activate

4. Installed the required libraries using pip
pip install langchain langchain-community langgraph langgraph-sdk openai pinecone faker python-dotenv huggingface_hub

5. Launched the development interface using:
jupyter notebook


This setup prepares the environment for building and running the LangGraph-based Travel-Sales Agent.





## 2. Create a Custom Travel Package Dataset using Faker and Random Logic

This cell generates a synthetic dataset of 5,000 unique travel packages using manually crafted logic and the `Faker` library.

Each record contains:
- A unique package title
- Destination 
- Descriptive summary
- Duration, price, and best time to visit
- Inclusions and exclusions
- Key tourist attractions

The dataset is randomized to reflect realistic travel package data, which will be used by the Travel and Sales agents later in the notebook.


In [95]:
import pandas as pd
import random

# Templates
package_templates = [
    "Romantic {place} Escape", "Adventure in {place}", "Cultural Wonders of {place}",
    "Luxury {place} Tour", "Budget-Friendly {place} Trip", "Wildlife Safari in {place}",
    "Historical {place} Expedition", "Relaxing {place} Retreat", "Scenic {place} Discovery"
]

# Places
global_places = [
    "Paris", "New York", "Tokyo", "Rome", "Istanbul", "Cape Town", "Bangkok", "Dubai",
    "Rio de Janeiro", "Barcelona", "Sydney", "Bali", "Prague", "Vienna", "Singapore", "Amsterdam",
    "Cairo", "Marrakech", "Reykjavik", "Zurich", "San Francisco", "Florence", "Budapest", "Lisbon"
]

# Place codes (e.g., Paris → PAR, New York → NY)
place_codes = {place: ''.join([w[0] for w in place.split()]).upper()[:3] for place in global_places}

# Themes and their short codes
themes = ["romantic", "adventurous", "cultural", "luxurious", "budget-friendly", "wildlife-focused", "historical", "relaxing", "scenic"]
theme_codes = {
    "romantic": "ROM", "adventurous": "ADV", "cultural": "CUL", "luxurious": "LUX",
    "budget-friendly": "BUD", "wildlife-focused": "WLD", "historical": "HIS",
    "relaxing": "REL", "scenic": "SCN"
}

features = [
    "ancient ruins", "breathtaking views", "local cuisine", "heritage sites", "vibrant nightlife",
    "nature trails", "mountain landscapes", "iconic landmarks", "coastal beauty"
]

attractions_pool = [
    "Eiffel Tower", "Central Park", "Tokyo Tower", "Colosseum", "Hagia Sophia", "Table Mountain",
    "Grand Palace", "Burj Khalifa", "Christ the Redeemer", "Sagrada Familia", "Opera House",
    "Ubud Rice Fields", "Prague Castle", "Schonbrunn Palace", "Gardens by the Bay", "Van Gogh Museum",
    "Pyramids of Giza", "Majorelle Garden", "Blue Lagoon", "Fresh Water Lake", "Golden Gate Bridge",
    "Duomo Cathedral", "Buda Castle", "Belem Tower", "Step Hills", "Three Sister Mountains"
]

inclusions_list = [
    "Flights, Hotel, Breakfast", "Train, Resort, All Meals", "Bus, Guided Tours, Dinner",
    "Flight, Camping, Trekking Gear", "Car, Palace Stay, Cultural Shows"
]

exclusions_list = [
    "Entry tickets, Lunch", "Alcoholic beverages, Personal expenses", "Tips, Travel Insurance",
    "Medical expenses, Extra activities", "Laundry, Telephone bills"
]

def create_description(place, theme, feature):
    return f"A {theme} experience in {place}, known for its {feature}. Ideal for travelers seeking unforgettable memories."

def generate_dataset(num_records):
    seen_codes = set()
    dataset = []

    while len(dataset) < num_records:
        place = random.choice(global_places)
        theme = random.choice(themes)
        feature = random.choice(features)
        template = random.choice(package_templates)

        theme_code = theme_codes[theme]
        place_code = place_codes[place]
        suffix = random.randint(1000, 9999)
        code = f"{theme_code}-{place_code}-{suffix}"

        if code in seen_codes:
            continue
        seen_codes.add(code)

        name = template.format(place=place)
        full_title = f"{name} ({code})"
        description = create_description(place, theme, feature)
        duration = f"{random.randint(2, 7)}N/{random.randint(3, 8)}D"
        price = random.randint(15000, 100000)
        attractions = ", ".join(random.sample(attractions_pool, 3))
        best_time = random.choice(['Winter', 'Summer', 'Spring', 'Autumn'])
        inclusions = random.choice(inclusions_list)
        exclusions = random.choice(exclusions_list)

        dataset.append({
            "Package Title": full_title,
            "Destination": place,
            "Description": description,
            "Duration": duration,
            "Price": price,
            "Attractions": attractions,
            "Best Time to Visit": best_time,
            "Inclusions": inclusions,
            "Exclusions": exclusions
        })

    return pd.DataFrame(dataset)

# Generate 5,000 records
df_preview = generate_dataset(5000)
df_preview.head(10)


,Package Title,Destination,Description,Duration,Price,Attractions,Best Time to Visit,Inclusions,Exclusions
0,Luxury Istanbul Tour (BUD-I-5859),Istanbul,"A budget-friendly experience in Istanbul, know...",7N/7D,66095,"Eiffel Tower, Central Park, Three Sister Mount...",Winter,"Flight, Camping, Trekking Gear","Laundry, Telephone bills"
1,Romantic Cape Town Escape (LUX-CT-1823),Cape Town,"A luxurious experience in Cape Town, known for...",6N/7D,48012,"Belem Tower, Duomo Cathedral, Opera House",Summer,"Car, Palace Stay, Cultural Shows","Entry tickets, Lunch"
2,Wildlife Safari in Tokyo (HIS-T-2335),Tokyo,"A historical experience in Tokyo, known for it...",3N/3D,42902,"Burj Khalifa, Golden Gate Bridge, Hagia Sophia",Winter,"Flights, Hotel, Breakfast","Laundry, Telephone bills"
3,Wildlife Safari in San Francisco (SCN-SF-8660),San Francisco,"A scenic experience in San Francisco, known fo...",4N/4D,83305,"Grand Palace, Colosseum, Prague Castle",Spring,"Flight, Camping, Trekking Gear","Medical expenses, Extra activities"
4,Relaxing Vienna Retreat (WLD-V-3713),Vienna,"A wildlife-focused experience in Vienna, known...",4N/4D,33603,"Central Park, Eiffel Tower, Grand Palace",Summer,"Bus, Guided Tours, Dinner","Laundry, Telephone bills"
5,Luxury Cairo Tour (ROM-C-2360),Cairo,"A romantic experience in Cairo, known for its ...",7N/8D,33422,"Christ the Redeemer, Schonbrunn Palace, Grand ...",Autumn,"Flight, Camping, Trekking Gear","Tips, Travel Insurance"
6,Cultural Wonders of Lisbon (SCN-L-6801),Lisbon,"A scenic experience in Lisbon, known for its i...",7N/8D,88855,"Central Park, Gardens by the Bay, Fresh Water ...",Autumn,"Flights, Hotel, Breakfast","Medical expenses, Extra activities"
7,Luxury Rio de Janeiro Tour (HIS-RDJ-4099),Rio de Janeiro,"A historical experience in Rio de Janeiro, kno...",5N/8D,68204,"Sagrada Familia, Golden Gate Bridge, Eiffel Tower",Summer,"Train, Resort, All Meals","Laundry, Telephone bills"
8,Relaxing Amsterdam Retreat (LUX-A-7399),Amsterdam,"A luxurious experience in Amsterdam, known for...",7N/3D,93651,"Three Sister Mountains, Blue Lagoon, Duomo Cat...",Spring,"Flights, Hotel, Breakfast","Entry tickets, Lunch"
9,Relaxing San Francisco Retreat (HIS-SF-1320),San Francisco,"A historical experience in San Francisco, know...",3N/7D,72399,"Pyramids of Giza, Eiffel Tower, Duomo Cathedral",Spring,"Train, Resort, All Meals","Tips, Travel Insurance"


## 3. Load Environment Variables

This step loads API keys securely from a `.env` file using the `python-dotenv` library.

- `OPENAI_API_KEY`: Used to access OpenAI's GPT models via LangChain
- `PINECONE_API_KEY`: Used to initialize and connect to the Pinecone vector database

Environment variables are accessed using `os.getenv()` after calling `load_dotenv()`.


In [96]:
from dotenv import load_dotenv
import os

# Load .env
load_dotenv()

# Access keys
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

## 4. Load Embedding Model using Hugging Face

In this step, we load the `all-MiniLM-L6-v2` model from Hugging Face Transformers using LangChain's `HuggingFaceEmbeddings` class.

This embedding model is used to convert travel package text data into vector representations for semantic similarity during retrieval.

Model used:
- `sentence-transformers/all-MiniLM-L6-v2` (lightweight and effective for dense retrieval tasks)


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


c:\Users\91832\Desktop\AI_TR_SA\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 5. Create Embeddings for the Dataset

In this step, a new column `combined_text` is created by merging multiple fields from each travel package into a single descriptive string.

This combined text is then passed through the Hugging Face embedding model to generate dense vector embeddings, which are stored in a new `embedding` column.

These embeddings are later used for semantic similarity-based retrieval during agent interaction.


In [ ]:
def format_combined_text(row):
    return (
        f"The package '{row['Package Title']}' is a {row['Duration']} trip to {row['Destination']} during {row['Best Time to Visit']}. "
        f"Description: {row['Description']}. "
        f"It includes {row['Inclusions']} and excludes {row['Exclusions']}. "
        f"Main attractions are {row['Attractions']}. Total cost is {row['Price']}."
    )

# 1. Create combined_text column
df_preview["combined_text"] = df_preview.apply(format_combined_text, axis=1)

# 2. Create embedding column (do NOT use inplace modification)
df_preview["embedding"] = df_preview["combined_text"].apply(embedding_model.embed_query)


## 6. Save the Embedded Dataset

The updated dataset, which now includes the `embedding` column, is saved to a `.pkl` (pickle) file named `df_with_embeddings.pkl`.

This file will be used later to load the embedded data during agent setup and retrieval.


In [ ]:
import pickle

with open("df_with_embeddings.pkl", "wb") as f:
    pickle.dump(df_preview, f)

print(" Saved to df_with_embeddings.pkl")


 Saved to df_with_embeddings.pkl


In [ ]:
print(f"Total embeddings generated: {len(df_preview['embedding'])}")


Total embeddings generated: 5000


In [ ]:
# Show the first embedding (as a sample)
print(df_preview['embedding'].iloc[0])


[0.03660408407449722, 0.005317443050444126, 0.019778789952397346, 0.033463578671216965, -0.04472091794013977, 0.01899327151477337, 0.00028995604952797294, 0.023420000448822975, 0.0016893295105546713, 0.008693227544426918, -0.023025216534733772, -0.044264987111091614, 0.02914135716855526, -0.001443033921532333, 0.00958577822893858, -0.016612162813544273, 0.05456189066171646, -0.012079059146344662, 0.12027633190155029, -0.09078435599803925, 0.022376731038093567, -0.017240794375538826, 0.06062893941998482, -0.04069354757666588, -0.03899528831243515, 0.10442976653575897, 0.06236394867300987, 0.003630194114521146, -0.005757179111242294, 0.022802604362368584, -0.02222175896167755, 0.11233393102884293, -0.028548767790198326, -0.08925475925207138, 0.0071049523539841175, 0.11299571394920349, -0.042454902082681656, -0.07269985973834991, 0.0007955258479341865, -0.03148362413048744, 0.05522187426686287, 0.019828055053949356, 0.07631043344736099, 0.013829939998686314, 0.02877691574394703, -0.023558

In [ ]:

print("Embedding format:", type(df_preview["embedding"].iloc[0]), len(df_preview["embedding"].iloc[0]))

Embedding format: <class 'list'> 384


## 7. Initialize and Create Pinecone Index

This step connects to the Pinecone vector database and creates a new index named `travel-packages` (if it doesn't already exist).

- API key is securely loaded from the `.env` file.
- The index is configured with:
  - 384-dimensional vectors (matching the embedding size)
  - Cosine similarity as the distance metric
  - AWS serverless environment (`us-east-1`)

Once created, the index is connected using `pc.Index()` for further upserts and queries.


In [ ]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

# Load from .env
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")

# Init client
pc = Pinecone(api_key=pinecone_api_key)

# Define index name and spec
index_name = "travel-packages"

# Create index (only once)
if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Connect to index
index = pc.Index(index_name)


## 8. Upload Embeddings and Metadata to Pinecone

This step loads the processed dataset from the saved pickle file and uploads vector embeddings along with relevant metadata to the Pinecone index.

- Each record includes:
  - A unique UUID
  - The embedding vector
  - Metadata such as package name, destination, description, price, duration, and other details

- The data is uploaded in batches of 100 vectors for efficiency.

Once uploaded, Pinecone will store and index the data for fast vector-based semantic search during chatbot interaction.


In [ ]:
import os
import uuid
import pickle
from tqdm import tqdm
from pinecone import Pinecone

# 1. Load the DataFrame with combined_text and embedding
with open("df_with_embeddings.pkl", "rb") as f:
    df_preview = pickle.load(f)

# 2. Connect to Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("travel-packages")  # Replace with your index name if different

# 3. Upload in batches
batch_size = 100
records = []

for i, row in tqdm(df_preview.iterrows(), total=len(df_preview)):
    metadata = {
        "text": row["combined_text"],  # Contains full structured summary
        "Package Name": row["Package Title"],
        "Destination": row["Destination"],
        "Description": row["Description"],
        "Duration": row["Duration"],
        "Price": int(row["Price"]),
        "Best Time to Visit": row["Best Time to Visit"],
        "Inclusions": row["Inclusions"],
        "Exclusions": row["Exclusions"]
    }

    record = {
        "id": str(uuid.uuid4()),             # Random unique ID
        "values": row["embedding"],          # The embedding vector
        "metadata": metadata                 # Useful for display & filtering
    }

    records.append(record)

    # Upload when batch is ready
    if len(records) == batch_size:
        index.upsert(vectors=records)
        records = []

# Final batch upload
if records:
    index.upsert(vectors=records)

print(" All vectors uploaded to Pinecone successfully!")


100%|██████████| 5000/5000 [07:55<00:00, 10.51it/s]

 All vectors uploaded to Pinecone successfully!


loading one vector to see  wheather the upsert is working or not 

In [ ]:

# Load one embedding vector to use as a query
query_vector = df_preview["embedding"][0]  # or from your .pkl

# Search Pinecone
response = index.query(
    vector=query_vector,
    top_k=1,
    include_metadata=True,
    include_values=True  #  this is key!
)

# See if values (embedding) are present
print(response)


{'matches': [{'id': '302b8cf3-7d09-4423-a11f-23ff6215d6c9',
              'metadata': {'Best Time to Visit': 'Winter',
                           'Description': 'A historical experience in Dubai, '
                                          'known for its iconic landmarks. '
                                          'Ideal for travelers seeking '
                                          'unforgettable memories.',
                           'Destination': 'Dubai',
                           'Duration': '6N/3D',
                           'Exclusions': 'Alcoholic beverages, Personal '
                                         'expenses',
                           'Inclusions': 'Flights, Hotel, Breakfast',
                           'Package Name': 'Scenic Dubai Discovery '
                                           '(HIS-D-4098)',
                           'Price': 85927.0,
                           'text': "The package 'Scenic Dubai Discovery "
                                   "(HIS

preview of 1st index embeddings,type and lenght.

In [ ]:
import pickle
with open("df_with_embeddings.pkl", "rb") as f:
    df_preview = pickle.load(f)


print(df_preview["embedding"].iloc[0])
print(type(df_preview["embedding"].iloc[0]))
print(len(df_preview["embedding"].iloc[0]))


[0.03660408407449722, 0.005317443050444126, 0.019778789952397346, 0.033463578671216965, -0.04472091794013977, 0.01899327151477337, 0.00028995604952797294, 0.023420000448822975, 0.0016893295105546713, 0.008693227544426918, -0.023025216534733772, -0.044264987111091614, 0.02914135716855526, -0.001443033921532333, 0.00958577822893858, -0.016612162813544273, 0.05456189066171646, -0.012079059146344662, 0.12027633190155029, -0.09078435599803925, 0.022376731038093567, -0.017240794375538826, 0.06062893941998482, -0.04069354757666588, -0.03899528831243515, 0.10442976653575897, 0.06236394867300987, 0.003630194114521146, -0.005757179111242294, 0.022802604362368584, -0.02222175896167755, 0.11233393102884293, -0.028548767790198326, -0.08925475925207138, 0.0071049523539841175, 0.11299571394920349, -0.042454902082681656, -0.07269985973834991, 0.0007955258479341865, -0.03148362413048744, 0.05522187426686287, 0.019828055053949356, 0.07631043344736099, 0.013829939998686314, 0.02877691574394703, -0.023558

In [ ]:
query = "What attractions are included in the Romantic Paris package?"
query_vector = embedding_model.embed_query(query)

results = index.query(vector=query_vector, top_k=10, include_metadata=True)


In [ ]:
for match in results['matches']:
    print("Score:", match['score'])
    print("Package:", match['metadata']['Package Name'])
    print("Destination:", match['metadata']['Destination'])
    print("---")


Score: 0.704474092
Package: Romantic Paris Escape (REL-P-2660)
Destination: Paris
---
Score: 0.703481138
Package: Romantic Paris Escape (ROM-P-5254)
Destination: Paris
---
Score: 0.700573266
Package: Romantic Paris Escape (ROM-P-9412)
Destination: Paris
---
Score: 0.699949
Package: Romantic Paris Escape (ROM-P-3394)
Destination: Paris
---
Score: 0.69940275
Package: Romantic Paris Escape (CUL-P-1109)
Destination: Paris
---
Score: 0.694697559
Package: Romantic Paris Escape (ROM-P-8591)
Destination: Paris
---
Score: 0.694526434
Package: Romantic Paris Escape (CUL-P-3608)
Destination: Paris
---
Score: 0.692383766
Package: Romantic Paris Escape (ROM-P-8595)
Destination: Paris
---
Score: 0.690094471
Package: Romantic Paris Escape (BUD-P-5165)
Destination: Paris
---
Score: 0.689149261
Package: Romantic Paris Escape (HIS-P-1448)
Destination: Paris
---


## 9. Set Up Travel Sales Consultant Retriever

In this step, the retriever for the Travel Consultant is configured.

- The Pinecone index is connected to LangChain's `PineconeVectorStore`.
- The `retriever` object allows for semantic search over the embedded dataset using user queries.
- The retriever will be passed into the Travel Conversational Chain to provide context-aware answers.

This setup enables the Travel Agent to fetch relevant packages based on similarity to user queries.


In [ ]:
# Step 1: Travel Consultant Retriever Setup
from langchain.vectorstores import Pinecone as PineconeVectorStore



# Load API key
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")




# Load vectorstore retriever
retriever = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model
).as_retriever()

print("Travel Consultant retriever is ready")


Travel Consultant retriever is ready


## 10. Define Prompt Templates for Travel and Sales Agents

In this step, prompt templates are created for the two specialized agents:

- **Travel Consultant Prompt**  
  Handles queries related to destinations, attractions, itineraries, and best time to visit.  
  Ignores or redirects questions about pricing, inclusions, or availability.

- **Sales Consultant Prompt**  
  Handles queries related to price, duration, inclusions, exclusions, and comparisons.  
  Avoids responding to travel-related topics and redirects such queries to the Travel Consultant.

Each prompt includes specific instructions to ensure clear separation of responsibilities and context-aware responses.


In [ ]:
from langchain.chains import  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# LLM (can adjust temperature if needed)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

travel_prompt = PromptTemplate.from_template("""
1) You are a helpful and knowledgeable Travel Consultant.

Your task is to answer user  questions about:
- travel destinations
- attractions
- sightseeing itineraries
- best time to visit specific places
-Only respond with what's explicitly asked. Avoid repeating other information from the context.

2) Do not answer questions about:
- price
- cost
- inclusions
- exclusions
- bookings
- availability
                                            

If the question is clearly about pricing, inclusions, or exclusions, reply only with:
"Please ask our Sales Consultant for pricing and detailed inclusions."


Travel Package Data:
{context}

User Query:
{question}
""")



sales_prompt = PromptTemplate.from_template("""
You are a helpful and knowledgeable Sales Consultant.

1) Your task is to answer questions related to:
- price
- cost
- duration
- inclusions
- exclusions
- package comparison by features
-Only respond with what's explicitly asked. Avoid repeating other information from the context.                                            
                                              
                                            
                                           

2) Do not answer questions that are specifically about:
- destinations
- attractions
- travel recommendations
- sightseeing itineraries

If the question is clearly about travel-related topics, respond only with:
"Please consult our Travel Consultant for recommendations about itineraries and destinations."

 .                                                                                        
                                            

Travel Package Data:
{context}

User Query:
{question}
""")







## 11. Create Conversational Chains for Travel and Sales Agents

This cell sets up `ConversationalRetrievalChain` for both the Travel and Sales Consultants.

- Each agent uses its own memory (`ConversationBufferMemory`) to retain previous conversation history.
- The same retriever (based on Pinecone) is used for both agents.
- Custom prompts (`travel_prompt` and `sales_prompt`) are passed to ensure domain-specific responses.

This setup enables the agents to provide context-aware answers grounded in the embedded travel package dataset.


In [ ]:
travel_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
sales_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 4. Chains — using combined_text as page_content (no need for metadata now)
travel_conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,  # from your Pinecone vector store setup
    memory=travel_memory,
    combine_docs_chain_kwargs={"prompt": travel_prompt}
)

sales_conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=sales_memory,
    combine_docs_chain_kwargs={"prompt": sales_prompt}
)

## 12. Test Travel and Sales Agents Separately

This cell performs a basic test to validate both the Travel and Sales Conversational Retrieval Chains.

- A sample query is passed to each agent individually.
- The Travel Consultant responds based on destinations and itineraries.
- The Sales Consultant is expected to redirect the query since it's outside its scope.

This test confirms that the prompt boundaries and chain logic are working as intended.


In [ ]:
question = "What are the Paris packages available?"

# Travel Consultant Test
response_travel = travel_conversational_chain.invoke({"question": question})
print("Travel Consultant:", response_travel["answer"])

# Sales Consultant Test
response_sales = sales_conversational_chain.invoke({"question": question})
print("Sales Consultant:", response_sales["answer"])


Travel Consultant: The Paris packages available are:
1) Budget-Friendly Paris Trip (ROM-P-2922)
2) Budget-Friendly Paris Trip (WLD-P-4471)
3) Budget-Friendly Paris Trip (ADV-P-7710)
4) Budget-Friendly Paris Trip (REL-P-4089)
Sales Consultant: Please consult our Travel Consultant for recommendations about itineraries and destinations.


In [ ]:
# Turn 1 – Travel Agent
response_1 = travel_conversational_chain.invoke({
    "question": "Tell me some good romantic destinations."
})
print("Travel Agent:", response_1["answer"])

# Turn 2 – Follow-up with Sales Agent
response_followup = sales_conversational_chain.invoke({
    "question": "What’s the price of Budget-Friendly Paris Trip (ROM-P-8634)"
})
print("Sales Agent:", response_followup["answer"])


Travel Agent: Some good romantic destinations are New York, Rio de Janeiro, and Lisbon.
Sales Agent: The price of the Budget-Friendly Paris Trip (ROM-P-8634) is 31467.


In [ ]:
# Your test query
question = "What is the price of Budget-Friendly Paris Trip (ROM-P-8634)"

# Get relevant docs using the sales agent's retriever
retrieved_docs = sales_conversational_chain.retriever.get_relevant_documents(question)

# Print the results
print(" Retrieved Documents:\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n{'-'*60}")


 Retrieved Documents:

Document 1:
The package 'Budget-Friendly Paris Trip (ROM-P-8634)' is a 5N/3D trip to Paris during Summer. Description: A romantic experience in Paris, known for its nature trails. Ideal for travelers seeking unforgettable memories.. It includes Train, Resort, All Meals and excludes Alcoholic beverages, Personal expenses. Main attractions are Buda Castle, Opera House, Tokyo Tower. Total cost is 31467.
------------------------------------------------------------
Document 2:
The package 'Budget-Friendly Paris Trip (ROM-P-8876)' is a 2N/8D trip to Paris during Spring. Description: A romantic experience in Paris, known for its coastal beauty. Ideal for travelers seeking unforgettable memories.. It includes Bus, Guided Tours, Dinner and excludes Entry tickets, Lunch. Main attractions are Step Hills, Majorelle Garden, Central Park. Total cost is 37663.
------------------------------------------------------------
Document 3:
The package 'Budget-Friendly Paris Trip (ROM-P

In [ ]:
print(travel_memory.chat_memory.messages)


[HumanMessage(content='What are the Paris packages available?', additional_kwargs={}, response_metadata={}), AIMessage(content='The Paris packages available are:\n1) Budget-Friendly Paris Trip (ROM-P-2922)\n2) Budget-Friendly Paris Trip (WLD-P-4471)\n3) Budget-Friendly Paris Trip (ADV-P-7710)\n4) Budget-Friendly Paris Trip (REL-P-4089)', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me some good romantic destinations.', additional_kwargs={}, response_metadata={}), AIMessage(content='Some good romantic destinations are New York, Rio de Janeiro, and Lisbon.', additional_kwargs={}, response_metadata={})]


## 13. Define Router Prompt for Agent Classification

In this step, a routing prompt is defined to classify incoming user queries into one of four categories:

- `travel`: questions related to destinations, sightseeing, best time to visit, etc.
- `sales`: questions about price, cost, inclusions, exclusions, etc.
- `multi`: if the query contains both travel and sales aspects (e.g., two-part questions)
- `fallback`: if the query is unrelated or unclear

This prompt is passed to a `LLMChain` (router_chain), which uses the OpenAI model to interpret and classify user intent. The result determines which agent will handle the query in the LangGraph flow.


In [ ]:
from langchain.chains.router import LLMRouterChain
from langchain.chains.llm import LLMChain

routing_prompt = PromptTemplate.from_template("""
You are a smart classifier. Classify the user query into one of four intents:

- "travel" → about destinations, attractions, best time to visit, sightseeing, city recommendations
- "sales" → about price, cost, inclusions, exclusions, cheapest, booking details
- "multi" → if it includes both the travel and sales topics or if a query has 2 or 3 sentences combine check clearly it is single intent or multi intent if it has both  intent then mark it as multi
- "fallback" → if unrelated to travel packages

Respond with only one of: travel, sales, multi, fallback.

User Question:
{question}
""")



router_chain = LLMChain(llm=llm, prompt=routing_prompt)


## 14 Test the Router Prompt

This cell tests the `router_chain` by running a set of sample queries through the routing LLM.

Each query is classified into one of the following categories: `travel`,`sales`,`multi`,`fallback`.

The classification result helps determine which agent should respond to each query.  
This step verifies that the router prompt is working correctly.


In [ ]:
test_queries = [
    "What’s the price of the Paris honeymoon trip?",
    "Suggest romantic destinations in Italy.",
    "Are meals included in the Tokyo trip?",
    "Tell me about things to do in Bali.",
    "How much and where should I go in June?",
    "what is happening in the world and how is you day ?"
]

for q in test_queries:
    decision = router_chain.run({"question": q}).strip().lower()
    print(f"Query: {q}")
    print(f" LLM Router Decision: {decision}\n")


Query: What’s the price of the Paris honeymoon trip?
 LLM Router Decision: sales

Query: Suggest romantic destinations in Italy.
 LLM Router Decision: travel

Query: Are meals included in the Tokyo trip?
 LLM Router Decision: sales

Query: Tell me about things to do in Bali.
 LLM Router Decision: travel

Query: How much and where should I go in June?
 LLM Router Decision: multi

Query: what is happening in the world and how is you day ?
 LLM Router Decision: fallback



## 15. Define Router Function for LangGraph

This cell defines the `route_agent()` function, which is responsible for classifying incoming user queries into one of the agent types:

- `"travel"` for travel-related queries
- `"sales"` for cost, inclusion, or duration queries
- `"multi"` for questions involving both travel and sales aspects
- `"fallback"` for queries that don't match any supported type

The function uses the output of `router_chain.invoke()` to determine the routing decision, which is used later in the LangGraph flow.


In [ ]:
from langgraph.graph import StateGraph, END
from langchain.memory import ConversationBufferMemory

# Create LangGraph state structure
def build_state():
    return {"chat_history": [], "question": "", "answer": ""}

# Memory assignment
travel_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
sales_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
travel_conversational_chain.memory = travel_memory
sales_conversational_chain.memory = sales_memory

# Helper Functions (Modular Reuse)
def get_travel_answer(question, chat_history):
    return travel_conversational_chain.invoke({
        "question": question,
        "chat_history": chat_history
    })["answer"]

def get_sales_answer(question, chat_history):
    return sales_conversational_chain.invoke({
        "question": question,
        "chat_history": chat_history
    })["answer"]

# Travel Node
def travel_node(state):
    print("\n[TRAVEL NODE]")
    print("Question:", state["question"])
    answer = get_travel_answer(state["question"], state.get("chat_history", []))
    state["answer"] = answer
    return state

# Sales Node
def sales_node(state):
    print("\n[SALES NODE]")
    print("Question:", state["question"])
    answer = get_sales_answer(state["question"], state.get("chat_history", []))
    state["answer"] = answer
    return state

# Multi Node
def multi_node(state):
    print("\n[MULTI NODE]")
    question = state["question"]
    chat_history = state.get("chat_history", [])

    travel_resp = get_travel_answer(question, chat_history)
    sales_resp = get_sales_answer(question, chat_history)

    # Ignore fallback-style replies
    if "please ask our sales consultant" in travel_resp.lower():
        travel_resp = ""
    if "please consult our travel consultant" in sales_resp.lower():
        sales_resp = ""

    if not travel_resp and not sales_resp:
        combined = (
            "I'm not sure how to help with that. You can ask about:\n"
            "- Travel destinations or attractions\n"
            "- Package prices, inclusions, or exclusions"
        )
    else:
        combined = (travel_resp + "\n\n" + sales_resp).strip()

    #print("Final Answer:", combined)
    state["answer"] = combined
    return state

# Fallback Node
def fallback_node(state):
    print("\n[FALLBACK NODE]")
    state["answer"] = (
        "I'm not sure how to help with that. You can ask about:\n"
        "- Travel destinations or attractions\n"
        "- Package prices, inclusions, or exclusions"
    )
    return state




## 16. Construct LangGraph Flow and Compile the Agent Router

This step defines the LangGraph flow that controls the conversation routing:

- Adds nodes for `travel`, `sales`, `multi`, and `fallback` agents
- Sets the router node as the entry point
- Uses `route_agent()` to dynamically decide which path to follow
- Adds conditional edges from the router to the appropriate agent
- Ends all agent paths with `END` to complete the interaction
- Finally, compiles the graph into `langgraph_router`

This setup enables a modular and intelligent conversation flow based on user intent.


In [ ]:
# Router function
def route_agent(state):
    result = router_chain.invoke({"question": state["question"]})
    route = result["text"].strip().lower()
    return route if route in ["travel", "sales", "multi"] else "fallback"

# Define the LangGraph flow
definition = StateGraph(state_schema=dict)
definition.add_node("travel", travel_node)
definition.add_node("sales", sales_node)
definition.add_node("multi", multi_node)
definition.add_node("fallback", fallback_node)
definition.set_entry_point("router")
definition.add_node("router", lambda state: {"next": route_agent(state), **state})
definition.add_conditional_edges("router", route_agent, {
    "travel": "travel",
    "sales": "sales",
    "multi": "multi",
    "fallback": "fallback",
})
definition.add_edge("travel", END)
definition.add_edge("sales", END)
definition.add_edge("multi", END)
definition.add_edge("fallback", END)

# Compile the router graph
langgraph_router = definition.compile()

## 17. Run the LangGraph Travel-Sales Agent

This is the main execution loop that activates the conversational agent.

- Displays a welcome message
- Waits for user input
- Builds a new LangGraph state with the user's question
- Invokes the compiled LangGraph router to route the question to the correct agent
- Prints the final answer from the appropriate agent(s)

The loop continues until the user types `exit` or `quit`, which ends the session.


In [ ]:

print("LangGraph Travel-Sales Agent is live. Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ["exit", "quit"]:
        print("Ending chat.")
        break

    state = build_state()
    state["question"] = user_input
    result = langgraph_router.invoke(state)

    print(f"\nFinal Answer: {result['answer']}\n")



LangGraph Travel-Sales Agent is live. Type 'exit' to quit.



[TRAVEL NODE]
Question: what are best destination in italy ?

Final Answer: Some of the best destinations in Italy are Rome, Florence, Venice, Milan, and the Amalfi Coast.


[TRAVEL NODE]
Question: what are the packages availble for rome ?

Final Answer: Some of the best destinations in Italy are Rome, Florence, Venice, Milan, and the Amalfi Coast.


[TRAVEL NODE]
Question: what are the packages available for rome ? 

Final Answer: The packages available for Rome are:
1) Budget-Friendly Rome Trip (ADV-R-1592)
2) Budget-Friendly Rome Trip (ROM-R-9076)
3) Budget-Friendly Rome Trip (ADV-R-6591)
4) Budget-Friendly Rome Trip (WLD-R-2153)


[SALES NODE]
Question: what is the duration For Budget-Friendly Rome Trip (ADV-R-1592)?

Final Answer: The duration of the Budget-Friendly Rome Trip (ADV-R-1592) is 2 nights and 6 days.


[SALES NODE]
Question: what is the price for it?

Final Answer: The total cost for the Budget-Friendly Rome Trip (ADV-R-1592) is 17889.


[SALES NODE]
Question: what ar